<a href="https://colab.research.google.com/github/stxupengyu/Yelp-Recomendation-Algorithms/blob/master/yelp_recomendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#link google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#enter the folder
!ls
import os
os.chdir('/content/drive/My Drive/colab/tourism-service-rec')
!ls

Dataset_Agreement.pdf		     yelp_academic_dataset_review.json
kaggle.json			     yelp_academic_dataset_tip.json
rating.csv			     yelp_academic_dataset_user.json
recommender-systems-kaggle.ipynb     yelp-data-analysis.ipynb
yelp_academic_dataset_business.json  yelp-recommendation.ipynb
yelp_academic_dataset_checkin.json
Dataset_Agreement.pdf		     yelp_academic_dataset_review.json
kaggle.json			     yelp_academic_dataset_tip.json
rating.csv			     yelp_academic_dataset_user.json
recommender-systems-kaggle.ipynb     yelp-data-analysis.ipynb
yelp_academic_dataset_business.json  yelp-recommendation.ipynb
yelp_academic_dataset_checkin.json


In [ ]:
#!pip install scikit-surprise
from surprise import SVD,SVDpp,NormalPredictor,NMF
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import KFold, split
import pandas as pd
# 指定文件所在路径
file_path = os.path.expanduser('rating.csv')
# 告诉文本阅读器，文本的格式是怎么样的
reader = Reader(line_format='user item rating', sep=',')
# 加载数据
data = Dataset.load_from_file(file_path, reader=reader)
#划分训练集测试集
train_s,test_s = split.train_test_split(data, train_size=0.8)
# 使用biasSVD
algo1 = SVD()
# 使用funkSVD
algo2 = SVD(biased = False)
# 使用SVD++
algo3 = SVDpp()
#使用纯随机
algo4=NormalPredictor()
#使用非负矩阵分解
algo5=NMF()

In [ ]:
from collections import defaultdict
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""
    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0
    return precisions, recalls

#data = Dataset.load_builtin('ml-100k')
#kf = KFold(n_splits=5)
#algo = SVD()
#for trainset, testset in kf.split(data):
    #algo.fit(trainset)
    #predictions = algo.test(testset)
    #precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4)
    # Precision and recall can then be averaged over all users
    #print(sum(prec for prec in precisions.values()) / len(precisions))
    #print(sum(rec for rec in recalls.values()) / len(recalls))
def result(algo,train_s,test_s,k,threshold):
  algo.fit(train_s)
  pre = algo.test(test_s)
  accuracy.rmse(pre,verbose=True)
  accuracy.mse(pre,verbose=True)
  accuracy.mae(pre,verbose=True)
  accuracy.fcp(pre,verbose=True)
  precisions, recalls = precision_recall_at_k(pre,k ,threshold)
  print('Precition:',sum(prec for prec in precisions.values()) / len(precisions))
  print('Recall:',sum(rec for rec in recalls.values()) / len(recalls))

In [ ]:
def all_result(k,threshold):  
  print('biasSVD结果')
  result(algo1,train_s,test_s,k,threshold)
  print('funkSVD结果')
  result(algo2,train_s,test_s,k,threshold)
  print('SVD++结果')
  result(algo3,train_s,test_s,k,threshold)
  print('随机结果')
  result(algo4,train_s,test_s,k,threshold)
  print('NMF结果')
  result(algo5,train_s,test_s,k,threshold)

In [ ]:
threshold=4
for k in [1,3,5,7,9]:
  print ("=================RESULT=======================")
  print('k=',k,'threshold=',threshold)
  print()
  all_result(k,threshold)

=================RESULT=======================
k= 1 threshold= 4

biasSVD结果
RMSE: 1.2247
MSE: 1.4999
MAE:  0.9946
FCP:  0.5767
Precition: 0.24396344396344397
Recall: 0.20956661370802784
funkSVD结果
RMSE: 2.2734
MSE: 5.1685
MAE:  1.9352
FCP:  0.5880
Precition: 0.014045214045214045
Recall: 0.007293994018380745
SVD++结果
RMSE: 1.2338
MSE: 1.5224
MAE:  1.0050
FCP:  0.5783
Precition: 0.23184223184223185
Recall: 0.19811074394046985
随机结果
RMSE: 1.7634
MSE: 3.1094
MAE:  1.4176
FCP:  0.5126
Precition: 0.2702260702260702
Recall: 0.23457127101427244
NMF结果
RMSE: 1.4077
MSE: 1.9818
MAE:  1.1441
FCP:  0.6024
Precition: 0.12265512265512266
Recall: 0.09350371716894078
=================RESULT=======================
k= 3 threshold= 4

biasSVD结果
RMSE: 1.2231
MSE: 1.4960
MAE:  0.9936
FCP:  0.5813
Precition: 0.24333814333814333
Recall: 0.2212242885114458
funkSVD结果
RMSE: 2.2588
MSE: 5.1024
MAE:  1.9187
FCP:  0.5769
Precition: 0.01537598204264871
Recall: 0.008461346787465115
SVD++结果
RMSE: 1.2317
MSE: 1.5172
MAE: 

In [ ]:
threshold=3.5
for k in [1,3,5,7,9]:
  print ("=================RESULT=======================")
  print('k=',k,'threshold=',threshold)
  print()
  all_result(k,threshold)

=================RESULT=======================
k= 1 threshold= 3.5

biasSVD结果
RMSE: 1.2240
MSE: 1.4983
MAE:  0.9947
FCP:  0.5772
Precition: 0.501010101010101
Recall: 0.44955786824473704
funkSVD结果
RMSE: 2.2572
MSE: 5.0948
MAE:  1.9210
FCP:  0.5739
Precition: 0.33275613275613275
Recall: 0.31764097027444793
SVD++结果
RMSE: 1.2315
MSE: 1.5166
MAE:  1.0026
FCP:  0.5800
Precition: 0.5033189033189033
Recall: 0.4514943983919454
随机结果
RMSE: 1.7711
MSE: 3.1368
MAE:  1.4203
FCP:  0.4888
Precition: 0.3657527657527658
Recall: 0.3225021256334388
NMF结果
RMSE: 1.4090
MSE: 1.9852
MAE:  1.1442
FCP:  0.5988
Precition: 0.5048581048581049
Recall: 0.4606795953765652
=================RESULT=======================
k= 3 threshold= 3.5

biasSVD结果
RMSE: 1.2248
MSE: 1.5001
MAE:  0.9956
FCP:  0.5744
Precition: 0.4907327240660575
Recall: 0.4861484007948654
funkSVD结果
RMSE: 2.2728
MSE: 5.1658
MAE:  1.9351
FCP:  0.5758
Precition: 0.3321308321308321
Recall: 0.32705396922568647
SVD++结果
RMSE: 1.2320
MSE: 1.5179
MAE:  1.0021
